In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np


In [8]:
data_6m = pd.read_csv('../../datas_after_tokenizer/6m/6m_after_prepro.csv')
data_7m = pd.read_csv('../../datas_after_tokenizer/7m/7m_after_prepro.csv')
data_8m = pd.read_csv('../../datas_after_tokenizer/8m/8m_after_prepro.csv')
data_9m = pd.read_csv('../../datas_after_tokenizer/9m/9m_after_prepro.csv')

In [9]:
df_6m = data_6m.sample(frac=1).reset_index(drop=True)
df_7m = data_7m.sample(frac=1).reset_index(drop=True)
df_8m = data_8m.sample(frac=1).reset_index(drop=True)
df_9m = data_9m.sample(frac=1).reset_index(drop=True)
print(df_6m.iloc[0:10,2])

0    1
1    0
2    0
3    0
4    0
5    1
6    1
7    0
8    3
9    1
Name: 1, dtype: int64


In [10]:
X_6m = df_6m.iloc[:, 1].values
y_6m = df_6m.iloc[:, 2].values
X_7m = df_7m.iloc[:, 1].values
y_7m = df_7m.iloc[:, 2].values
X_8m = df_8m.iloc[:, 1].values
y_8m = df_8m.iloc[:, 2].values
X_9m = df_9m.iloc[:, 1].values
y_9m = df_9m.iloc[:, 2].values

In [11]:
X_train_text_6m, X_test_text_6m, y_train_6m, y_test_6m = train_test_split(X_6m, y_6m, test_size=0.2)
X_train_text_7m, X_test_text_7m, y_train_7m, y_test_7m = train_test_split(X_7m, y_7m, test_size=0.2)
X_train_text_8m, X_test_text_8m, y_train_8m, y_test_8m = train_test_split(X_8m, y_8m, test_size=0.2)
X_train_text_9m, X_test_text_9m, y_train_9m, y_test_9m = train_test_split(X_9m, y_9m, test_size=0.2)

# 아니다 여기서는 합치는게 맞는 것 같다. numpy 

https://stackoverflow.com/questions/21887754/numpy-concatenate-two-arrays-vertically

참고해서. 왜냐하면 tf-idf값을 추출하려면 훈련 단계별로 접근하는데 여기서 4번 훈련하면 4단계로 나뉘어져 있어서 문서 전체의 값을 뽑기 힘들어서. 만약 이 방법이 있으면 합치지 않아도 됌.

http://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.W7ycdmgzZPY

https://stackoverflow.com/questions/25217510/how-to-see-top-n-entries-of-term-document-matrix-after-tfidf-in-scikit-learn

이 글이 좋음. 이걸로 해보자.


In [12]:
print(len(X_train_text_6m))
print(X_train_text_6m[0])
print(y_train_6m[:10])

161144
서울시 교육감 후보 공명선거 다짐 오류 우회 위한 함수 추가 지방 선거 서울 교육감 선거 출마 조희연 박선영 조영달 후보 왼쪽 일 서울 상암동 후보 초청 토론회 앞서 손 공명선거 다짐 국회 사진 기자단 세상 눈 글로벌 미디어 세계일보 
[0 0 3 3 0 0 0 2 3 0]


In [13]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {
    'logisticregression__C' : [0.001, 0.01, 0.1, 1, 10]
}

grid = GridSearchCV(pipe, param_grid, cv=5)
#print(y_train.shape)
grid.fit(X_train_text_6m, y_train_6m)
grid.fit(X_train_text_7m, y_train_7m)
grid.fit(X_train_text_8m, y_train_8m)
grid.fit(X_train_text_9m, y_train_9m)
print("최상의 교차 검증 점수 : %.2f" %(grid.best_score_))

최상의 교차 검증 점수 : 0.97


In [14]:
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_6m, y_test_6m)))
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_7m, y_test_7m)))
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_8m, y_test_8m)))
print(" 테스트 점수 : %.2f" %(grid.score(X_test_text_9m, y_test_9m)))

 테스트 점수 : 0.98
 테스트 점수 : 0.98
 테스트 점수 : 0.98
 테스트 점수 : 0.97


In [15]:
print('done')

done


In [16]:
vector = grid.best_estimator_.named_steps["tfidfvectorizer"]

In [17]:
indices = np.argsort(vector.idf_)[::-1]
features = vector.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
print(top_features)

['장식장', '합삭']


In [18]:
#특성 이름을 구한다.
X_train_6m = vector.transform(X_train_text_6m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_6m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("6월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("6월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])


6월 tfidf가 가장 낮은 특성 20 개 : 
 ['양승택' '나훔' '나훗카항' '구회장' '낙관론' '샘스미스' '연녹색' '연노랑' '구호법' '복강' '구호대' '구호금'
 '보호처분' '인시디어스' '최학진' '라에스' '총무처' '라시나' '연기장' '낙법']
6월 tfidf가 가장 높은 특성 20 개 : 
  ['김준희' '탁현민' '공증' '가구' '젤라' '쉐어링' '수경' '타이어' '코딩' '김광석' '진세연' '머크' '수박'
 '프로젝터' '손지현' '암웨이' '복권' '틀니' '육우' '티맥스']


In [19]:
m_6m =  feature_names[sorted_by_tfidf[-3000:]]

In [20]:

#특성 이름을 구한다.
X_train_7m = vector.transform(X_train_text_7m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_7m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("7월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("7월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("7월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


7월 tfidf가 가장 낮은 특성 20 개 : 
 ['실예' '공배' '김준래' '야간열차' '불자동차' '신재천' '맹성' '휘둥그레진' '케로' '불연속' '맹수호' '어혈'
 '어네스토' '이삼수' '맹종' '신정수' '이상도' '앵클부츠' '장미계' '정낙균']
7월 tfidf가 가장 높은 특성 20 개 : 
  ['골판지' '김광석' '진세연' '육회' '보드게임' '정미홍' '가구' '프로젝터' '김옥빈' '손지현' '타이어' '저글링'
 '암웨이' '복권' '보노보노' '루돌프' '육우' '틀니' '아보카도' '티맥스']
7월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '함께' '네이버' '대한' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '이후' '기사' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '문제' '연합뉴스' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '해야' '필요' '대통령' '모두' '자신' '현재' '언론' '발표' '영상' '기업' '세계' '가장'
 '지역' '코리아' '따라' '올해' '경우' '결과' '다양' '국내' '가운데' '공식' '사업' '지난해' '직접' '지원'
 '적극' '주요']


In [21]:
m_7m =  feature_names[sorted_by_tfidf[-3000:]]

In [22]:

#특성 이름을 구한다.
X_train_8m = vector.transform(X_train_text_8m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_8m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("8월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("8월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("8월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


8월 tfidf가 가장 낮은 특성 20 개 : 
 ['강희주' '대장기' '박찬일' '만월제' '변온' '길문종' '원격탐사' '컬사' '박무택' '길로이' '장군봉' '변상익'
 '신정수' '각장' '이종록' '이종매' '옥예' '군역' '락탐' '방효창']
8월 tfidf가 가장 높은 특성 20 개 : 
  ['가구' '전차' '프로젝터' '키자니아' '손지현' '와인' '결핵' '강예빈' '스팀' '게놈' '복권' '보노보노' '지부'
 '모나미' '로이킴' '루돌프' '틀니' '아보카도' '육우' '티맥스']
8월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '함께' '네이버' '대한' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '이후' '기사' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '문제' '연합뉴스' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '해야' '필요' '대통령' '모두' '자신' '현재' '언론' '발표' '영상' '기업' '세계' '가장'
 '지역' '코리아' '따라' '올해' '경우' '결과' '다양' '국내' '가운데' '공식' '사업' '지난해' '직접' '지원'
 '적극' '주요']


In [23]:
m_8m = feature_names[sorted_by_tfidf[-3000:]]

In [24]:

#특성 이름을 구한다.
X_train_9m = vector.transform(X_train_text_9m)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_9m.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()

print("9월 tfidf가 가장 낮은 특성 20 개 : \n", feature_names[sorted_by_tfidf[:20]])
print("9월 tfidf가 가장 높은 특성 20 개 : \n ", feature_names[sorted_by_tfidf[-20:]])

sorted_by_idf = np.argsort(vector.idf_)
print("9월 idf가 가장 낮은 특성 100개 \n", feature_names[sorted_by_idf[:100]])


9월 tfidf가 가장 낮은 특성 20 개 : 
 ['율리시스' '주변광' '악필' '작구' '금천역' '관악역' '도봉역' '김춘추' '조석구' '발작성' '가톨릭의료원' '정낙균'
 '엄기성' '김동집' '방광경' '서호경' '마려워' '유문부' '윤동섭' '조기진']
9월 tfidf가 가장 높은 특성 20 개 : 
  ['결핵' '신협' '베리' '타이어' '저글링' '스팀' '암웨이' '제대혈' '지부' '복권' '보노보노' '방귀' '모나미'
 '로이킴' '루돌프' '아이폰' '틀니' '아보카도' '육우' '티맥스']
9월 idf가 가장 낮은 특성 100개 
 ['기자' '금지' '배포' '무단' '위한' '추가' '오류' '우회' '함수' '전재' '뉴스' '서울' '사진' '이미지'
 '오후' '원본' '경제' '통해' '저작권자' '지난' '한다' '위해' '바로' '채널' '함께' '네이버' '대한' '한국'
 '이날' '제보' '페이스북' '이번' '된다' '진행' '구독' '열린' '대표' '제공' '방송' '시간' '가능' '관련'
 '시장' '시작' '이후' '기사' '예정' '공개' '대해' '이상' '사람' '우리' '정부' '설명' '모습' '확인'
 '특히' '한편' '최근' '미국' '참석' '스타' '관계자' '문제' '연합뉴스' '정보' '이야기' '상황' '지방' '계획'
 '선거' '오늘' '해야' '필요' '대통령' '모두' '자신' '현재' '언론' '발표' '영상' '기업' '세계' '가장'
 '지역' '코리아' '따라' '올해' '경우' '결과' '다양' '국내' '가운데' '공식' '사업' '지난해' '직접' '지원'
 '적극' '주요']


In [25]:
m_9m = feature_names[sorted_by_tfidf[-3000:]]

In [26]:
tf_idf_json = {}
for data in [m_6m, m_7m, m_8m, m_9m]:
    for words in data:
        if words in tf_idf_json:
            print(words, " 키값 있음! ")
        else:
            tf_idf_json[words] = 1
            
print(len(tf_idf_json))


당갈  키값 있음! 
닉스  키값 있음! 
이능룡  키값 있음! 
최강희  키값 있음! 
김이나  키값 있음! 
윤성호  키값 있음! 
김정은  키값 있음! 
박원빈  키값 있음! 
미나  키값 있음! 
조리  키값 있음! 
스파이더맨  키값 있음! 
빅스비  키값 있음! 
김슬기  키값 있음! 
스피커  키값 있음! 
경차  키값 있음! 
표창  키값 있음! 
폭스  키값 있음! 
등기  키값 있음! 
쪽지  키값 있음! 
알집  키값 있음! 
티웨이항공  키값 있음! 
고창군  키값 있음! 
오리진  키값 있음! 
고성희  키값 있음! 
어음  키값 있음! 
소포  키값 있음! 
사춘기  키값 있음! 
앤디  키값 있음! 
드라마  키값 있음! 
이엘  키값 있음! 
하랑  키값 있음! 
유재석  키값 있음! 
에보  키값 있음! 
스컬  키값 있음! 
허민  키값 있음! 
은경  키값 있음! 
플레디스  키값 있음! 
이천희  키값 있음! 
지진희  키값 있음! 
허진호  키값 있음! 
전자서명  키값 있음! 
하하  키값 있음! 
유이  키값 있음! 
함영  키값 있음! 
김지석  키값 있음! 
정주리  키값 있음! 
노안  키값 있음! 
김영옥  키값 있음! 
이혜정  키값 있음! 
보미  키값 있음! 
번역가  키값 있음! 
이윤미  키값 있음! 
연구비  키값 있음! 
원유  키값 있음! 
카이저  키값 있음! 
택시  키값 있음! 
세르비아  키값 있음! 
실망  키값 있음! 
압기  키값 있음! 
인증  키값 있음! 
프릭  키값 있음! 
멕시카나  키값 있음! 
학익  키값 있음! 
델타  키값 있음! 
김연아  키값 있음! 
균주  키값 있음! 
플라잉  키값 있음! 
프롤  키값 있음! 
증권  키값 있음! 
패스  키값 있음! 
컬투쇼  키값 있음! 
스퍼  키값 있음! 
현대모비스  키값 있음! 
염동열  키값 있음! 
조정민  키값 있음! 
도진  키값 있음! 
결재  키값 있음! 
설빙  키값 있음! 
고아라  키값 있음! 
임원식

정화  키값 있음! 
김진우  키값 있음! 
두산  키값 있음! 
이케아  키값 있음! 
노치  키값 있음! 
페루  키값 있음! 
박준형  키값 있음! 
정은채  키값 있음! 
부역  키값 있음! 
빗소리  키값 있음! 
큐브  키값 있음! 
김장미  키값 있음! 
삼계탕  키값 있음! 
김기덕  키값 있음! 
제니  키값 있음! 
아바타  키값 있음! 
음료  키값 있음! 
인공지능  키값 있음! 
농구  키값 있음! 
한복  키값 있음! 
전력량계  키값 있음! 
유진  키값 있음! 
소니  키값 있음! 
리벳  키값 있음! 
위니  키값 있음! 
문지애  키값 있음! 
대만  키값 있음! 
소진  키값 있음! 
어카운트  키값 있음! 
이수근  키값 있음! 
지진  키값 있음! 
정읍  키값 있음! 
함소원  키값 있음! 
독전  키값 있음! 
보훈  키값 있음! 
박재민  키값 있음! 
김현주  키값 있음! 
랜드  키값 있음! 
귀농  키값 있음! 
알베르토  키값 있음! 
금보  키값 있음! 
력사  키값 있음! 
쌀보리  키값 있음! 
카리오  키값 있음! 
레옹  키값 있음! 
빕스  키값 있음! 
프린세스  키값 있음! 
후기  키값 있음! 
꽈배기  키값 있음! 
뉴본  키값 있음! 
서유리  키값 있음! 
런닝맨  키값 있음! 
정경호  키값 있음! 
한예리  키값 있음! 
스카  키값 있음! 
마틸다  키값 있음! 
자연인  키값 있음! 
모르타르  키값 있음! 
슐츠  키값 있음! 
스코  키값 있음! 
연비  키값 있음! 
서장훈  키값 있음! 
태양광  키값 있음! 
금융  키값 있음! 
신탁  키값 있음! 
박명수  키값 있음! 
서윤아  키값 있음! 
구례  키값 있음! 
김종국  키값 있음! 
노트북  키값 있음! 
북아현  키값 있음! 
이상우  키값 있음! 
주파수  키값 있음! 
주성치  키값 있음! 
이필운  키값 있음! 
보수  키값 있음! 
송강호  키값 있음! 
비스킷  키값 있음! 
목성  키값 있음! 
결핵  

송하윤  키값 있음! 
포켓몬  키값 있음! 
양배추  키값 있음! 
젤라  키값 있음! 
로봇  키값 있음! 
하연수  키값 있음! 
박보람  키값 있음! 
원전  키값 있음! 
김교순  키값 있음! 
손태영  키값 있음! 
수박  키값 있음! 
한서희  키값 있음! 
양파  키값 있음! 
조재현  키값 있음! 
일품진로  키값 있음! 
김준희  키값 있음! 
탁현민  키값 있음! 
김현중  키값 있음! 
쉐어링  키값 있음! 
수경  키값 있음! 
동아일보  키값 있음! 
김광석  키값 있음! 
진세연  키값 있음! 
가구  키값 있음! 
프로젝터  키값 있음! 
손지현  키값 있음! 
타이어  키값 있음! 
암웨이  키값 있음! 
복권  키값 있음! 
육우  키값 있음! 
틀니  키값 있음! 
아보카도  키값 있음! 
티맥스  키값 있음! 
레드햇  키값 있음! 
해밍턴  키값 있음! 
백지영  키값 있음! 
시하  키값 있음! 
영양제  키값 있음! 
문화재  키값 있음! 
보육원  키값 있음! 
오리온  키값 있음! 
디원  키값 있음! 
던전  키값 있음! 
윤창중  키값 있음! 
진구  키값 있음! 
이마트  키값 있음! 
장기하  키값 있음! 
세아  키값 있음! 
중립  키값 있음! 
수퍼  키값 있음! 
명함  키값 있음! 
윔블던  키값 있음! 
클루니  키값 있음! 
페르노리카  키값 있음! 
부지사  키값 있음! 
안구건조증  키값 있음! 
달력  키값 있음! 
행정관  키값 있음! 
단양  키값 있음! 
국세  키값 있음! 
한기웅  키값 있음! 
유비  키값 있음! 
몬스터  키값 있음! 
강훈  키값 있음! 
리벳  키값 있음! 
히라마  키값 있음! 
두산엔진  키값 있음! 
나명  키값 있음! 
디자이너  키값 있음! 
윤희재  키값 있음! 
교화소  키값 있음! 
피벗  키값 있음! 
아이언맨  키값 있음! 
워크맨  키값 있음! 
테마주  키값 있음! 
이닝  키값 있음! 
김준  키값 있음! 
인쇄  키값 있음! 
페르소나

난초  키값 있음! 
영준  키값 있음! 
카지노  키값 있음! 
쿠바  키값 있음! 
캐나다  키값 있음! 
서리  키값 있음! 
매스  키값 있음! 
할머니  키값 있음! 
치타  키값 있음! 
박은빈  키값 있음! 
방수  키값 있음! 
민도희  키값 있음! 
실리콘  키값 있음! 
조타  키값 있음! 
러빈  키값 있음! 
영화관  키값 있음! 
정진철  키값 있음! 
전수진  키값 있음! 
유가령  키값 있음! 
허경환  키값 있음! 
박수인  키값 있음! 
윤서인  키값 있음! 
박정아  키값 있음! 
외환  키값 있음! 
직위  키값 있음! 
지원금  키값 있음! 
치약  키값 있음! 
건선  키값 있음! 
김규리  키값 있음! 
배터리  키값 있음! 
어택  키값 있음! 
기한  키값 있음! 
야마하  키값 있음! 
이시원  키값 있음! 
줄리엔  키값 있음! 
탈모  키값 있음! 
강주은  키값 있음! 
로드먼  키값 있음! 
존슨  키값 있음! 
김현정  키값 있음! 
이동욱  키값 있음! 
손흥민  키값 있음! 
세연  키값 있음! 
테메르  키값 있음! 
남보원  키값 있음! 
청렴도  키값 있음! 
진진  키값 있음! 
정은지  키값 있음! 
다이소  키값 있음! 
오재호  키값 있음! 
옥주현  키값 있음! 
김서룡  키값 있음! 
무크  키값 있음! 
박재현  키값 있음! 
전력  키값 있음! 
탄자니아  키값 있음! 
김현우  키값 있음! 
암웨이  키값 있음! 
서강준  키값 있음! 
정준호  키값 있음! 
호날두  키값 있음! 
옴부즈만  키값 있음! 
이시영  키값 있음! 
거미  키값 있음! 
레종  키값 있음! 
이카루스  키값 있음! 
원산  키값 있음! 
오나미  키값 있음! 
옥분  키값 있음! 
텐센트  키값 있음! 
이혼  키값 있음! 
육성재  키값 있음! 
부영  키값 있음! 
아궁  키값 있음! 
족발  키값 있음! 
김슬기  키값 있음! 
일급비밀  키값 있음! 
한복  키값 있음! 
임상시험  키값 있음!

황정음  키값 있음! 
정유미  키값 있음! 
레깅스  키값 있음! 
곱창  키값 있음! 
도박  키값 있음! 
수세미  키값 있음! 
가정부  키값 있음! 
이소윤  키값 있음! 
정려원  키값 있음! 
헌혈  키값 있음! 
조용필  키값 있음! 
솔빈  키값 있음! 
턱관절  키값 있음! 
백종원  키값 있음! 
유로  키값 있음! 
솔비  키값 있음! 
오영주  키값 있음! 
윤세아  키값 있음! 
홍종현  키값 있음! 
아일라  키값 있음! 
변정수  키값 있음! 
아이스크림  키값 있음! 
김유정  키값 있음! 
장희진  키값 있음! 
박광현  키값 있음! 
슬리피  키값 있음! 
와이브로  키값 있음! 
지드래곤  키값 있음! 
카레  키값 있음! 
조윤희  키값 있음! 
장윤정  키값 있음! 
신협  키값 있음! 
한국학  키값 있음! 
개헌  키값 있음! 
체리  키값 있음! 
구혜선  키값 있음! 
혜리  키값 있음! 
청년  키값 있음! 
송백경  키값 있음! 
교촌  키값 있음! 
자급  키값 있음! 
자이언티  키값 있음! 
스테파니  키값 있음! 
유채영  키값 있음! 
윤아  키값 있음! 
세레스  키값 있음! 
증여  키값 있음! 
페트리  키값 있음! 
편의점  키값 있음! 
캐딜락  키값 있음! 
디기  키값 있음! 
강소봉  키값 있음! 
윤정수  키값 있음! 
송하윤  키값 있음! 
오랑우탄  키값 있음! 
금연  키값 있음! 
사무엘  키값 있음! 
꼰대  키값 있음! 
최은경  키값 있음! 
유인영  키값 있음! 
양미  키값 있음! 
바이낸스  키값 있음! 
민화  키값 있음! 
로꼬  키값 있음! 
민지영  키값 있음! 
미스코리아  키값 있음! 
추사랑  키값 있음! 
오스  키값 있음! 
허각  키값 있음! 
정세운  키값 있음! 
존박  키값 있음! 
박은지  키값 있음! 
망고  키값 있음! 
담배  키값 있음! 
미란다  키값 있음! 
강혜진  키값 있음! 
발목  키값 있음! 
지창욱  키값 있음! 
강승윤 

김혜연  키값 있음! 
지도  키값 있음! 
닭발  키값 있음! 
케로  키값 있음! 
이혼  키값 있음! 
인증서  키값 있음! 
서리  키값 있음! 
넥타  키값 있음! 
태풍  키값 있음! 
한은정  키값 있음! 
육성재  키값 있음! 
나혜미  키값 있음! 
족발  키값 있음! 
김슬기  키값 있음! 
일급비밀  키값 있음! 
한복  키값 있음! 
임상시험  키값 있음! 
거래소  키값 있음! 
오트밀  키값 있음! 
진드기  키값 있음! 
민원  키값 있음! 
박해미  키값 있음! 
부엉이  키값 있음! 
김호영  키값 있음! 
벌점  키값 있음! 
심소  키값 있음! 
자우림  키값 있음! 
판상  키값 있음! 
축제  키값 있음! 
미투  키값 있음! 
환불  키값 있음! 
정액  키값 있음! 
알로에  키값 있음! 
공찬  키값 있음! 
이지아  키값 있음! 
치빈  키값 있음! 
하석진  키값 있음! 
최동열  키값 있음! 
배지현  키값 있음! 
김준호  키값 있음! 
김남주  키값 있음! 
혜림  키값 있음! 
고은  키값 있음! 
프렌즈  키값 있음! 
개코  키값 있음! 
농약  키값 있음! 
쇼핑백  키값 있음! 
전교조  키값 있음! 
홍여진  키값 있음! 
풍동  키값 있음! 
조승희  키값 있음! 
총영사  키값 있음! 
게임빌  키값 있음! 
오설록  키값 있음! 
박유나  키값 있음! 
국정원  키값 있음! 
치질  키값 있음! 
임현준  키값 있음! 
문항  키값 있음! 
스톡옵션  키값 있음! 
기념일  키값 있음! 
초밥  키값 있음! 
바둑  키값 있음! 
교육  키값 있음! 
히치콕  키값 있음! 
달러  키값 있음! 
셀바스  키값 있음! 
블루홀  키값 있음! 
김민경  키값 있음! 
수송기  키값 있음! 
아프리카  키값 있음! 
퇴직금  키값 있음! 
우조  키값 있음! 
라돈  키값 있음! 
방제  키값 있음! 
알리바바  키값 있음! 
우표첩  키값 있음! 
허경환  키값 있음! 
오하영  키값 있음! 
귀

서민정  키값 있음! 
루나  키값 있음! 
종자  키값 있음! 
고등어  키값 있음! 
센서  키값 있음! 
위스키  키값 있음! 
어비스  키값 있음! 
왓츠  키값 있음! 
류화영  키값 있음! 
박지빈  키값 있음! 
다일  키값 있음! 
하이모  키값 있음! 
추성훈  키값 있음! 
버섯  키값 있음! 
김재우  키값 있음! 
소각  키값 있음! 
정인  키값 있음! 
김아중  키값 있음! 
임지연  키값 있음! 
강승윤  키값 있음! 
롯데  키값 있음! 
우주  키값 있음! 
장근석  키값 있음! 
강동호  키값 있음! 
냉매  키값 있음! 
강성태  키값 있음! 
보육  키값 있음! 
오만석  키값 있음! 
구재이  키값 있음! 
리니지  키값 있음! 
장나라  키값 있음! 
계단  키값 있음! 
오라클  키값 있음! 
육수  키값 있음! 
혜박  키값 있음! 
현영  키값 있음! 
택시  키값 있음! 
사유리  키값 있음! 
헤나  키값 있음! 
설현  키값 있음! 
성소  키값 있음! 
클레르  키값 있음! 
동우  키값 있음! 
이효리  키값 있음! 
이소연  키값 있음! 
오페라  키값 있음! 
박상민  키값 있음! 
김수민  키값 있음! 
유재명  키값 있음! 
민호  키값 있음! 
산업단지  키값 있음! 
소쿠리  키값 있음! 
카페  키값 있음! 
캡스  키값 있음! 
손예진  키값 있음! 
얼음  키값 있음! 
아이패드  키값 있음! 
한혜진  키값 있음! 
조현우  키값 있음! 
김민교  키값 있음! 
홍현희  키값 있음! 
배정남  키값 있음! 
효민  키값 있음! 
루스  키값 있음! 
오정연  키값 있음! 
김광규  키값 있음! 
세리  키값 있음! 
마리아  키값 있음! 
클라우드  키값 있음! 
유승우  키값 있음! 
방탄소년단  키값 있음! 
장윤정  키값 있음! 
리디  키값 있음! 
요금제  키값 있음! 
다시마  키값 있음! 
황승언  키값 있음! 
피츠버그  키값 있음! 
손여은  키값 있음! 
배유진  키값 있

In [27]:
import json

with open('./tf_idf_json.json', 'w') as f:
    json.dump(tf_idf_json, f)

In [28]:
with open('./tf_idf_json.json', 'r') as f:
    data = json.load(f)